In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('energydata_complete.csv')

In [55]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [56]:
from sklearn.linear_model import LinearRegression

In [57]:
lr = LinearRegression()

In [58]:
X = df[['T2']]

In [59]:
y = df['T6']

In [60]:
lr.fit(X,y)

LinearRegression()

In [61]:
from sklearn.metrics import r2_score

In [62]:
score = r2_score(y, lr.predict(X))

In [63]:
round(score, 2)

0.64

In [64]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [65]:
new_df = df.drop(columns=['date', 'lights'])

In [66]:
normalized_df = pd.DataFrame(scaler.fit_transform(new_df), columns=new_df.columns)

In [67]:
X =normalized_df.drop(columns=['Appliances'])
y = normalized_df['Appliances']

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 42 )

In [70]:
mlr = LinearRegression()
mlr.fit(X_train,y_train)

LinearRegression()

In [71]:
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(y_test, mlr.predict(X_test))

In [72]:
score

0.05013362397742959

In [73]:
rss = np.sum(np.square(y_test - mlr.predict(X_test)))

In [74]:
round(rss, 2)

45.35

In [75]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, mlr.predict(X_test)))
round(rmse, 3)

0.088

In [76]:
r2 = r2_score(y_test, mlr.predict(X_test))
round(r2, 2)

0.15

In [77]:
def get_weights_df(model, feat, col_name) :
#this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = [ 'Features' , col_name]
  weights_df[col_name].round( 3 )
  return weights_df

linear_model_weights = get_weights_df(mlr, X_train, 'Linear_Model_Weight' )

In [78]:
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [79]:
from sklearn.linear_model import Ridge

In [80]:
rid = Ridge(alpha=0.4)
rid.fit(X_train, y_train)

Ridge(alpha=0.4)

In [81]:
rmse = np.sqrt(mean_squared_error(y_test, rid.predict(X_test)))
round(rmse, 3)

0.088

In [43]:
from sklearn.linear_model import Lasso
las = Lasso(alpha=0.001)
las.fit(X_train, y_train)

Lasso(alpha=0.001)

In [44]:
lasso_model_weights = get_weights_df(las, X_train, 'Linear_Model_Weight' )

In [50]:
lasso_model_weights

,Features,Linear_Model_Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [49]:
lasso_model_weights[lasso_model_weights['Linear_Model_Weight'] != 0].shape

(4, 2)

In [51]:
rmse = np.sqrt(mean_squared_error(y_test, las.predict(X_test)))
round(rmse, 3)

0.094